# 6.2 Dependency Management and Package Distribution

This notebook demonstrates best practices for managing Python dependencies and distributing PySpark code in Databricks environments.

## Learning Objectives
- Understand dependency management strategies in Databricks
- Learn to package PySpark modules for distribution
- Master library installation patterns
- Create wheel packages for functional modules
- Implement version control for dependencies
- Build reusable, distributable PySpark libraries

In [ ]:
# For local development: Uncomment the next line
# %run 00_Environment_Setup.ipynb

## Why Dependency Management Matters

Proper dependency management is critical for production PySpark applications:

**Key Challenges**:
1. **Reproducibility**: Ensure code runs identically across environments
2. **Version Conflicts**: Manage conflicting library versions
3. **Distribution**: Deploy code across Spark cluster nodes
4. **Governance**: Control library usage in production
5. **Performance**: Minimize overhead from dependency loading

**Functional Programming Alignment**:
- Well-managed dependencies enable pure function libraries to be shared
- Versioned packages ensure deterministic behavior
- Proper distribution ensures transformations execute consistently across nodes

## Databricks Library Installation Patterns

Databricks supports multiple methods for installing Python libraries:

In [ ]:
print("=== Databricks Library Installation Methods ===")

installation_methods = """
1. CLUSTER-SCOPED LIBRARIES
   - Installed on entire cluster
   - Available to all notebooks and jobs on that cluster
   - Requires cluster restart
   - Best for: Shared dependencies across team
   
   Installation:
   - UI: Cluster > Libraries > Install New
   - API: Databricks CLI or REST API
   
   Use Cases:
   - Common data science libraries (pandas, numpy, scikit-learn)
   - Stable, production dependencies
   - Team-wide standards

2. NOTEBOOK-SCOPED LIBRARIES (%pip, %conda)
   - Installed in specific notebook session only
   - No cluster restart required
   - Isolated from other notebooks
   - Best for: Experimentation, notebook-specific needs
   
   Installation:
   %pip install <package-name>
   %pip install -r requirements.txt
   
   Use Cases:
   - Exploratory analysis
   - Testing new libraries
   - Temporary dependencies

3. WORKSPACE FILES
   - Upload Python files (.py, .whl) to workspace
   - Import as modules in notebooks
   - Version controlled through workspace
   - Best for: Internal utility modules
   
   Installation:
   - Upload via UI or Databricks CLI
   - Import: sys.path.append('/Workspace/...')
   
   Use Cases:
   - Custom utility functions
   - Internal libraries
   - Shared code modules

4. UNITY CATALOG VOLUMES (Recommended for Production)
   - Centralized library storage with governance
   - Access control and auditing
   - Version management
   - Best for: Production dependencies
   
   Installation:
   %pip install /Volumes/catalog/schema/volume/package.whl
   
   Use Cases:
   - Production-grade libraries
   - Governance requirements
   - Centralized dependency management

5. PYPI (Public Python Package Index)
   - Install from PyPI repository
   - Largest ecosystem of Python packages
   - Best for: Public, well-maintained libraries
   
   Installation:
   %pip install package-name==version
   
   Use Cases:
   - Standard libraries (requests, boto3, etc.)
   - Popular data science tools
   - Open-source dependencies
"""

print(installation_methods)

## Notebook-Scoped Library Installation

Demonstrate notebook-scoped installation with %pip:

In [ ]:
# ✅ BEST PRACTICE: Notebook-scoped installation for isolated dependencies

# Example: Install specific version for reproducibility
# %pip install pandas==2.0.0

# Example: Install from requirements file
# %pip install -r requirements.txt

# Example: Install multiple packages
# %pip install requests boto3 pyyaml

# Example: Install from Unity Catalog volume
# %pip install /Volumes/main/libraries/python/my_package-1.0.0-py3-none-any.whl

print("=== Notebook-Scoped Installation Best Practices ===")
print("\n✅ DO:")
print("  - Pin exact versions (package==1.2.3) for reproducibility")
print("  - Use requirements.txt for complex dependencies")
print("  - Install at the beginning of notebook")
print("  - Document why each library is needed")
print("  - Use %pip (not !pip) in Databricks")

print("\n❌ DON'T:")
print("  - Install in the middle of notebook execution")
print("  - Use unpinned versions (package without version)")
print("  - Install unnecessary dependencies")
print("  - Mix %pip and !pip commands")
print("  - Install different versions in different cells")

## Creating Reusable Python Modules

Package functional PySpark code into reusable modules:

In [ ]:
print("=== Creating Reusable PySpark Modules ===")

# Example: Module structure for reusable transformations
module_example = '''
# File: pyspark_transformations/cleaning.py
"""
Data cleaning transformations for PySpark.
All functions are pure - DataFrame in, DataFrame out.
"""

from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from typing import List

__version__ = "1.0.0"
__author__ = "Your Team"

def remove_nulls(df: DataFrame, columns: List[str]) -> DataFrame:
    """
    Remove rows with null values in specified columns.
    
    Pure function - no side effects.
    
    Args:
        df: Input DataFrame
        columns: List of column names to check for nulls
        
    Returns:
        DataFrame with null rows removed
        
    Example:
        >>> clean_df = remove_nulls(raw_df, ["id", "amount"])
    """
    for col in columns:
        df = df.filter(F.col(col).isNotNull())
    return df

def standardize_column_names(df: DataFrame) -> DataFrame:
    """
    Standardize column names to lowercase with underscores.
    
    Pure function - idempotent transformation.
    
    Args:
        df: Input DataFrame
        
    Returns:
        DataFrame with standardized column names
    """
    for col_name in df.columns:
        new_name = col_name.lower().replace(" ", "_").replace("-", "_")
        if new_name != col_name:
            df = df.withColumnRenamed(col_name, new_name)
    return df

def filter_date_range(df: DataFrame, date_column: str, 
                     start_date: str, end_date: str) -> DataFrame:
    """
    Filter DataFrame to date range.
    
    Pure function with configurable parameters.
    
    Args:
        df: Input DataFrame
        date_column: Name of date column
        start_date: Start date (YYYY-MM-DD format)
        end_date: End date (YYYY-MM-DD format)
        
    Returns:
        Filtered DataFrame
    """
    return df.filter(
        (F.col(date_column) >= F.lit(start_date)) &
        (F.col(date_column) <= F.lit(end_date))
    )
'''

print("Example reusable module:")
print(module_example)

print("\n✅ Module Best Practices:")
print("  - Clear docstrings with examples")
print("  - Type hints for all parameters")
print("  - Version information (__version__)")
print("  - Pure functions only")
print("  - Comprehensive error handling")
print("  - Unit tests for each function")

## Building Python Wheel Packages

Create distributable wheel packages for PySpark libraries:

In [ ]:
print("=== Building Python Wheel Packages ===")

# Project structure for wheel package
package_structure = """
pyspark_transformations/
│
├── setup.py                      # Package configuration
├── README.md                     # Package documentation
├── LICENSE                       # License file
├── requirements.txt              # Runtime dependencies
├── requirements-dev.txt          # Development dependencies
│
├── pyspark_transformations/      # Package directory
│   ├── __init__.py              # Package initialization
│   ├── __version__.py           # Version information
│   ├── cleaning.py              # Data cleaning module
│   ├── business_logic.py        # Business transformation module
│   ├── analytics.py             # Analytics module
│   └── utils.py                 # Utility functions
│
└── tests/                        # Test directory
    ├── __init__.py
    ├── test_cleaning.py
    ├── test_business_logic.py
    └── test_analytics.py
"""

print("Package structure:")
print(package_structure)

# setup.py example
setup_py_example = '''
# setup.py
from setuptools import setup, find_packages

# Read version from __version__.py
version = {}
with open("pyspark_transformations/__version__.py") as f:
    exec(f.read(), version)

# Read dependencies from requirements.txt
with open("requirements.txt") as f:
    requirements = f.read().splitlines()

# Read long description from README
with open("README.md", "r", encoding="utf-8") as f:
    long_description = f.read()

setup(
    name="pyspark-transformations",
    version=version["__version__"],
    author="Your Team",
    author_email="team@example.com",
    description="Reusable PySpark transformation library",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/yourorg/pyspark-transformations",
    packages=find_packages(exclude=["tests", "tests.*"]),
    install_requires=requirements,
    python_requires=">=3.8",
    classifiers=[
        "Development Status :: 4 - Beta",
        "Intended Audience :: Developers",
        "Topic :: Software Development :: Libraries",
        "License :: OSI Approved :: MIT License",
        "Programming Language :: Python :: 3",
        "Programming Language :: Python :: 3.8",
        "Programming Language :: Python :: 3.9",
        "Programming Language :: Python :: 3.10",
    ],
    keywords="pyspark spark data-engineering etl",
    project_urls={
        "Bug Reports": "https://github.com/yourorg/pyspark-transformations/issues",
        "Source": "https://github.com/yourorg/pyspark-transformations",
    },
)
'''

print("\nsetup.py example:")
print(setup_py_example)

# Build commands
build_commands = """
# Building wheel package

# 1. Install build tools
pip install build wheel setuptools

# 2. Build the wheel
python -m build

# This creates:
# dist/pyspark_transformations-1.0.0-py3-none-any.whl
# dist/pyspark_transformations-1.0.0.tar.gz

# 3. Install locally for testing
pip install dist/pyspark_transformations-1.0.0-py3-none-any.whl

# 4. Upload to Databricks (Unity Catalog volume)
databricks fs cp dist/pyspark_transformations-1.0.0-py3-none-any.whl \
  dbfs:/Volumes/main/libraries/python/

# 5. Install in Databricks notebook
%pip install /Volumes/main/libraries/python/pyspark_transformations-1.0.0-py3-none-any.whl
"""

print("\nBuild and deployment commands:")
print(build_commands)

## Dependency Versioning and Pinning

Proper version management ensures reproducibility:

In [ ]:
print("=== Dependency Versioning Strategies ===")

requirements_examples = """
# requirements.txt - Different versioning strategies

# ✅ BEST: Exact version pinning (most reproducible)
pyspark==3.4.1
pandas==2.0.3
pyarrow==12.0.1

# ✅ GOOD: Compatible version (allows patch updates)
requests~=2.31.0    # Allows 2.31.x but not 2.32.0
boto3~=1.28.0

# ⚠️  ACCEPTABLE: Minimum version (less safe)
numpy>=1.24.0
scikit-learn>=1.3.0

# ❌ AVOID: No version constraint (not reproducible)
# pandas
# requests

# ✅ GOOD: Version range (balanced approach)
matplotlib>=3.7.0,<4.0.0

# ✅ GOOD: Exclude problematic versions
sqlalchemy>=1.4.0,!=1.4.42,<2.0.0

# Development dependencies (requirements-dev.txt)
pytest==7.4.0
pytest-cov==4.1.0
black==23.7.0
ruff==0.0.280
"""

print(requirements_examples)

print("\n✅ Versioning Best Practices:")
print("\n1. Production Dependencies:")
print("   - Use exact pinning (==) for critical dependencies")
print("   - Document why each version is chosen")
print("   - Test before upgrading versions")
print("   - Keep security patches current")

print("\n2. Development Dependencies:")
print("   - Separate requirements-dev.txt file")
print("   - Pin testing framework versions")
print("   - Update tooling regularly")

print("\n3. Version Updates:")
print("   - Use dependabot or similar tools")
print("   - Review changelogs before upgrading")
print("   - Test in dev/staging before production")
print("   - Document breaking changes")

print("\n4. Conflict Resolution:")
print("   - Use pip-compile for dependency resolution")
print("   - Document conflicts and resolutions")
print("   - Consider using virtual environments")

## Using Requirements Files in Databricks

Demonstrate requirements file management in notebooks:

In [ ]:
print("=== Requirements File Management ===")

# Create example requirements file
requirements_content = """# Production dependencies for revenue analytics pipeline
# Updated: 2024-01-15

# Core dependencies
pyspark==3.4.1
pandas==2.0.3
pyarrow==12.0.1

# Data validation
great-expectations==0.17.23
pydantic==2.4.0

# Utilities
python-dateutil==2.8.2
pyyaml==6.0.1

# AWS integration (if needed)
boto3==1.28.57
"""

# Write to temporary file (for demonstration)
import tempfile
import os

with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as f:
    f.write(requirements_content)
    temp_req_file = f.name

print("Example requirements.txt:")
print(requirements_content)

# In Databricks, you would install with:
# %pip install -r /Workspace/path/to/requirements.txt
# or
# %pip install -r /Volumes/catalog/schema/volume/requirements.txt

print("\n✅ Installation in Databricks:")
print("  # From workspace file:")
print("  %pip install -r /Workspace/Repos/my-project/requirements.txt")
print("")
print("  # From Unity Catalog volume (recommended):")
print("  %pip install -r /Volumes/main/config/dependencies/requirements.txt")
print("")
print("  # From local file (for testing):")
print("  %pip install -r requirements.txt")

# Clean up
os.unlink(temp_req_file)

print("\n✅ Benefits of requirements files:")
print("  - Version control for dependencies")
print("  - Reproducible environments")
print("  - Easy collaboration")
print("  - Simplified deployment")
print("  - Documentation of dependencies")

## Importing and Using Custom Modules

Demonstrate importing custom PySpark modules in notebooks:

In [ ]:
print("=== Importing Custom Modules ===")

import_examples = '''
# ===================================================
# METHOD 1: Import from workspace file
# ===================================================

import sys
sys.path.append('/Workspace/Repos/my-project/src')

from pyspark_transformations import cleaning
from pyspark_transformations import business_logic

# Use the imported functions
clean_df = cleaning.remove_nulls(raw_df, ["id", "amount"])
result_df = business_logic.calculate_revenue(clean_df)

# ===================================================
# METHOD 2: Import from installed wheel package
# ===================================================

# First install the package
%pip install /Volumes/main/libraries/python/pyspark_transformations-1.0.0-py3-none-any.whl

# Then import normally
from pyspark_transformations import cleaning
from pyspark_transformations import business_logic
from pyspark_transformations import analytics

# ===================================================
# METHOD 3: Import from Databricks Repos
# ===================================================

# Add repo path to Python path
import sys
sys.path.append('/Workspace/Repos/user@example.com/my-pyspark-project/src')

# Import modules
from extraction import readers
from transformation import cleaning, business_logic
from loading import writers

# ===================================================
# METHOD 4: Use %run for notebook modules
# ===================================================

# Run another notebook to import its functions
%run ./modules/cleaning_functions
%run ./modules/business_logic_functions

# Now use functions defined in those notebooks
clean_df = clean_transactions(raw_df)
result_df = calculate_metrics(clean_df)

# ===================================================
# BEST PRACTICE: Package initialization
# ===================================================

# __init__.py in package root
from .cleaning import remove_nulls, standardize_column_names
from .business_logic import calculate_tax, calculate_revenue
from .analytics import revenue_by_category
from .__version__ import __version__

__all__ = [
    "remove_nulls",
    "standardize_column_names",
    "calculate_tax",
    "calculate_revenue",
    "revenue_by_category",
    "__version__",
]

# Then import is simpler:
from pyspark_transformations import remove_nulls, calculate_revenue
'''

print(import_examples)

print("\n✅ Import Best Practices:")
print("  - Use wheel packages for production code")
print("  - Keep import paths consistent across team")
print("  - Use __init__.py for clean imports")
print("  - Document import methods in README")
print("  - Avoid circular imports")
print("  - Use relative imports within packages")

## Governance and Security Considerations

Best practices for secure dependency management:

In [ ]:
print("=== Governance and Security ===")

governance_guidelines = """
1. LIBRARY APPROVAL PROCESS
   
   ✅ Before using external libraries:
   - Security scan for known vulnerabilities
   - License compatibility check
   - Maintenance status verification
   - Performance impact assessment
   - Team approval for production use
   
   Tools:
   - safety check (vulnerability scanning)
   - pip-audit (dependency auditing)
   - GitHub Dependabot (automated alerts)

2. UNITY CATALOG VOLUMES (Recommended)
   
   ✅ Advantages:
   - Centralized library storage
   - Access control and auditing
   - Version management
   - Compliance tracking
   - Team collaboration
   
   Setup:
   CREATE VOLUME main.libraries.python;
   
   Grant access:
   GRANT READ FILES ON VOLUME main.libraries.python TO data_engineers;

3. SECURITY SCANNING
   
   Regular vulnerability checks:
   
   # Scan dependencies for vulnerabilities
   pip install safety
   safety check -r requirements.txt
   
   # Audit with pip-audit
   pip install pip-audit
   pip-audit -r requirements.txt

4. LICENSE COMPLIANCE
   
   Track dependency licenses:
   
   # List licenses
   pip install pip-licenses
   pip-licenses
   
   Approved licenses:
   - MIT, Apache 2.0, BSD (permissive)
   
   Avoid:
   - GPL (copyleft, may require source disclosure)
   - Unknown/custom licenses

5. PRIVATE PACKAGE REPOSITORIES
   
   For proprietary libraries:
   
   # Install from private PyPI
   %pip install --extra-index-url https://your-pypi.example.com/simple/ \
        your-private-package==1.0.0
   
   # Or use Unity Catalog volumes
   %pip install /Volumes/main/libraries/python/your_package-1.0.0-py3-none-any.whl

6. ENVIRONMENT ISOLATION
   
   Development vs Production:
   
   - Development: Notebook-scoped installations for experimentation
   - Staging: Cluster libraries matching production
   - Production: Locked versions, approved libraries only

7. DEPENDENCY MONITORING
   
   Continuous monitoring:
   
   - GitHub Dependabot alerts
   - Snyk vulnerability scanning
   - Regular dependency updates
   - Security advisory subscriptions
"""

print(governance_guidelines)

print("\n✅ Security Checklist:")
print("  □ All dependencies scanned for vulnerabilities")
print("  □ Licenses reviewed and approved")
print("  □ Versions pinned in requirements.txt")
print("  □ Unity Catalog volumes used for production libraries")
print("  □ Access controls configured")
print("  □ Monitoring and alerting enabled")
print("  □ Update process documented")
print("  □ Rollback plan in place")

## Complete Example: Building and Deploying a Package

End-to-end workflow for creating and deploying a PySpark package:

In [ ]:
print("=== Complete Package Deployment Workflow ===")

workflow = """
STEP 1: PROJECT SETUP
=====================

# Create project structure
mkdir -p pyspark_revenue_analytics/pyspark_revenue_analytics
mkdir -p pyspark_revenue_analytics/tests
cd pyspark_revenue_analytics

# Initialize git
git init
git remote add origin https://github.com/yourorg/pyspark-revenue-analytics.git

# Create virtual environment
python -m venv venv
source venv/bin/activate  # On Windows: venv\\Scripts\\activate


STEP 2: DEVELOP MODULES
=======================

# Create module files
# pyspark_revenue_analytics/__init__.py
# pyspark_revenue_analytics/cleaning.py
# pyspark_revenue_analytics/business_logic.py
# pyspark_revenue_analytics/analytics.py

# Write unit tests
# tests/test_cleaning.py
# tests/test_business_logic.py


STEP 3: CREATE PACKAGE CONFIGURATION
====================================

# Create setup.py (shown earlier)
# Create __version__.py
echo '__version__ = "1.0.0"' > pyspark_revenue_analytics/__version__.py

# Create requirements.txt
cat > requirements.txt << EOF
pyspark>=3.4.0
pandas>=2.0.0
pyarrow>=12.0.0
EOF

# Create requirements-dev.txt
cat > requirements-dev.txt << EOF
pytest>=7.4.0
pytest-cov>=4.1.0
black>=23.7.0
ruff>=0.0.280
EOF


STEP 4: TEST LOCALLY
====================

# Install dev dependencies
pip install -r requirements-dev.txt

# Run tests
pytest tests/ -v --cov=pyspark_revenue_analytics

# Format code
black pyspark_revenue_analytics/

# Lint code
ruff check pyspark_revenue_analytics/


STEP 5: BUILD PACKAGE
=====================

# Install build tools
pip install build wheel setuptools

# Build wheel package
python -m build

# Output:
# dist/pyspark_revenue_analytics-1.0.0-py3-none-any.whl
# dist/pyspark_revenue_analytics-1.0.0.tar.gz

# Test installation
pip install dist/pyspark_revenue_analytics-1.0.0-py3-none-any.whl


STEP 6: DEPLOY TO DATABRICKS
============================

# Option A: Upload to Unity Catalog volume (RECOMMENDED)
databricks fs cp dist/pyspark_revenue_analytics-1.0.0-py3-none-any.whl \
  dbfs:/Volumes/main/libraries/python/

# Grant access
# In SQL:
# GRANT READ FILES ON VOLUME main.libraries.python TO data_engineers;

# Option B: Upload to workspace
databricks workspace import dist/pyspark_revenue_analytics-1.0.0-py3-none-any.whl \
  /Workspace/Shared/libraries/


STEP 7: INSTALL IN NOTEBOOKS
============================

# In Databricks notebook:
%pip install /Volumes/main/libraries/python/pyspark_revenue_analytics-1.0.0-py3-none-any.whl

# Import and use
from pyspark_revenue_analytics import cleaning, business_logic, analytics

clean_df = cleaning.remove_nulls(raw_df, ["id"])
revenue_df = business_logic.calculate_revenue(clean_df)
summary_df = analytics.revenue_by_category(revenue_df)


STEP 8: VERSION UPDATES
=======================

# Update version
echo '__version__ = "1.1.0"' > pyspark_revenue_analytics/__version__.py

# Rebuild and redeploy
python -m build
databricks fs cp dist/pyspark_revenue_analytics-1.1.0-py3-none-any.whl \
  dbfs:/Volumes/main/libraries/python/

# Update installations
%pip install --upgrade /Volumes/main/libraries/python/pyspark_revenue_analytics-1.1.0-py3-none-any.whl


STEP 9: CI/CD AUTOMATION
========================

# .github/workflows/build-and-deploy.yml
name: Build and Deploy

on:
  push:
    tags:
      - 'v*'

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.10'
      - name: Install dependencies
        run: |
          pip install build wheel
      - name: Run tests
        run: |
          pip install -r requirements-dev.txt
          pytest
      - name: Build package
        run: python -m build
      - name: Deploy to Databricks
        run: |
          databricks fs cp dist/*.whl dbfs:/Volumes/main/libraries/python/
        env:
          DATABRICKS_HOST: ${{ secrets.DATABRICKS_HOST }}
          DATABRICKS_TOKEN: ${{ secrets.DATABRICKS_TOKEN }}
"""

print(workflow)

print("\n✅ Deployment Best Practices:")
print("  - Use semantic versioning (MAJOR.MINOR.PATCH)")
print("  - Tag releases in git")
print("  - Maintain CHANGELOG.md")
print("  - Test in development environment first")
print("  - Use CI/CD for automated deployment")
print("  - Keep multiple versions for rollback")
print("  - Document upgrade paths")

## Summary

**Key Takeaways:**

1. **Dependency Management Strategies**:
   - Cluster-scoped for shared dependencies
   - Notebook-scoped for experimentation
   - Unity Catalog volumes for production
   - Requirements files for reproducibility

2. **Package Development**:
   - Modular structure with setup.py
   - Wheel packages for distribution
   - Comprehensive testing
   - Clear documentation

3. **Version Control**:
   - Exact pinning for production
   - Semantic versioning
   - Requirements files in version control
   - Regular security updates

4. **Security and Governance**:
   - Vulnerability scanning
   - License compliance
   - Access controls
   - Monitoring and auditing

5. **Deployment Workflow**:
   - Local development and testing
   - Package building
   - Upload to Unity Catalog
   - Installation in notebooks
   - CI/CD automation

**Best Practices for Dependency Management**:
- Use Unity Catalog volumes for production libraries
- Pin dependency versions for reproducibility
- Scan for vulnerabilities regularly
- Document dependencies and their purpose
- Automate package building and deployment
- Maintain separate dev and production environments
- Test upgrades thoroughly before production

**Functional Programming Benefits**:
- Pure function libraries are easily packaged and distributed
- Versioned packages ensure deterministic transformations
- Modular design enables reusable components
- Proper dependencies make functional code portable across projects

This completes the PySpark Best Practices curriculum. You now have the knowledge to build production-grade, functional PySpark applications with proper modular design and dependency management!

## Exercise

Practice dependency management and packaging:

1. Create a requirements.txt with pinned versions for a sample project
2. Build a simple Python package with setup.py
3. Create a wheel package using python -m build
4. Write installation instructions for Databricks
5. Plan a CI/CD workflow for automated package deployment
6. Document security scanning procedures
7. Create a version update strategy

In [ ]:
# Your exercise code here

# 1. Create requirements.txt content
your_requirements = """
# Your project dependencies
"""

# 2. Design setup.py structure
your_setup_py = """
# Your setup.py configuration
"""

# 3. Plan deployment workflow
your_deployment_plan = """
# Your step-by-step deployment process
"""

# 4. Document security procedures
your_security_checklist = """
# Your security scanning and approval process
"""

# 5. Design CI/CD workflow
your_cicd_workflow = """
# Your automated build and deployment workflow
"""

# Print your plans
# print(your_requirements)
# print(your_setup_py)
# print(your_deployment_plan)